In [1]:
! pip install torch
! pip install torch_geometric

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import RGCNConv

class RGCN(nn.Module):
    def __init__(self, in_feat, hidden_feat, out_feat, num_rels):
        super(RGCN, self).__init__()
        self.rgcn1 = RGCNConv(in_feat, hidden_feat, num_rels)
        self.rgcn2 = RGCNConv(hidden_feat, out_feat, num_rels)

    def forward(self, data):
        x, edge_index, edge_type = data.x, data.edge_index, data.edge_attr

        x = self.rgcn1(x, edge_index, edge_type)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.rgcn2(x, edge_index, edge_type)

        return x

In [3]:
# Create a small heterogeneous graph
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
edge_type = torch.tensor([0, 1, 0, 1], dtype=torch.long)
x = torch.randn(3, 10)  # 3 nodes with 10 features each

data = Data(x=x, edge_index=edge_index, edge_attr=edge_type)

# Initialize the R-GCN model
model = RGCN(in_feat=10, hidden_feat=16, out_feat=8, num_rels=2)

# Forward pass
output = model(data)

# Print the output node features
print(output)

tensor([[ 0.1019, -0.3130, -1.3179,  2.1006, -0.0242,  2.6194, -0.0906, -1.2258],
        [ 0.1092,  0.5595, -1.5334, -0.7592, -4.5224,  2.7065, -0.5410,  1.4654],
        [ 1.6849, -1.6635, -2.2862,  0.9605, -0.8661,  3.0515, -0.9478,  2.3525]],
       grad_fn=<AddBackward0>)


In [4]:
# define loss function and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [5]:
# TODO: define real target labels
target = torch.tensor([0, 1, 0], dtype=torch.long) 

In [6]:
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    output = model(data)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 2.5195772647857666
Epoch 10, Loss: 3.362293004989624
Epoch 20, Loss: 2.1101956367492676
Epoch 30, Loss: 1.9376786947250366
Epoch 40, Loss: 2.263345956802368
Epoch 50, Loss: 1.6783949136734009
Epoch 60, Loss: 0.9202739596366882
Epoch 70, Loss: 1.4042404890060425
Epoch 80, Loss: 1.0002321004867554
Epoch 90, Loss: 0.6702699065208435


In [7]:
# TODO: Define the target labels
target = torch.tensor([0, 1, 0], dtype=torch.long)
output = model(data)

loss = loss_fn(output, target)
print(loss)
# train the model with your data
#model.train()

tensor(0.8927, grad_fn=<NllLossBackward0>)
